# Méthodes itératives pour les systèmes linéaires

Les méthodes itératives donnent, en théorie, la solution d&#8217;un système linéaire après un nombre infini d&#8217;itérations.

Une méthode itérative s&#8217;appuie sur les étapes basiques suivantes:


$$
\begin{align*}
\mathbf{x}^{(0)} &   \text{ donné} &  \qquad \text{(initialisation)}\\
\mathbf{r^{(k)}} & = \mathbf{b} - A\mathbf{x}^{(k)} & \qquad \text{(résidu)}\\
P \mathbf{d}^{(k)} & = \mathbf{r}^{(k)}  & \qquad \text{(direction de descente)}\\
\mathbf{x}^{(k+1)} & =  \mathbf{x}^{(k)} + \mathbf{d}^{(k)} & \qquad \text{(itération)}
\end{align*}
$$

À chaque itération, elles nécessitent le calcul du résidu du système, de la direction de descente et de la nouvelle solution approchée.

*Note:* Dans le cas d&#8217;une matrice pleine, leur coût est donc de l&#8217;ordre de $n^2$ opérations à chaque itération, alors que le coût des méthodes directes est, en tout et pour tout, de l&#8217;ordre de $2n^3/3$.
Les méthodes itératives peuvent donc devenir ***compétitives*** si elles convergent en un nombre d&#8217;itérations indépendant de $n$, ou croissant sous-linéairement avec $n$.
Côut du calcul (flops) et de la memoire occupée (bytes): on considère la méthode de Cholesky et du gradient conjugué pour des matrices creuses de taille $n$ (qui dérivent de l&#8217;approximation des solutions de l&#8217;équation de Poisson par la méthode des éléments finis) avec $m$ éléments non nuls.

|  |  | Cholesky | Gradient Conjugué | flops(Chol.)/flops(GC) | Mem(Chol.)/Mem(GC) |
|  |  | -------- | ----------------- | ---------------------- | ------------------ |
| n | m / n^2 | flops | Memoire | flops | Memoire |  |  |
| 47 | 0.12 | 8.05 e+03 | 464 | 1.26 e+04 | 228 | 0.64 | 2.04 |
| 83 | 0.07 | 3.96 e+04 | 1406 | 3.03 e+04 | 533 | 1.31 | 2.64 |
| 150 | 0.04 | 2.01 e+05 | 4235 | 8.86 e+04 | 1245 | 2.26 | 3.4 |
| 225 | 0.03 | 6.39 e+05 | 9260 | 1.95 e+05 | 2073 | 3.27 | 4.47 |
| 329 | 0.02 | 1.74 e+06 | 17974 | 3.39 e+05 | 3330 | 5.15 | 5.39 |
| 424 | 0.02 | 3.78 e+06 | 30815 | 5.49 e+05 | 4513 | 6.88 | 6.83 |
| 530 | 0.01 | 8.31 e+06 | 50785 | 8.61 e+05 | 5981 | 9.65 | 8.49 |
| 661 | 0.01 | 1.19 e+07 | 68468 | 1.11 e+06 | 7421 | 10.66 | 9.23 |


*Important:* Pour les grandes matrice creuses, les méthodes directes vues dans les chapitre 2 s&#8217;avèrent parfois très coûteuses à cause du remplissage et les méthodes itératives peuvent offrir une alternative intéressante.

*Note:* On remarque que lorsque le conditionnement de la matrice devient très grand, la solution obtenue par la méthode directe peut être mauvaise mauvaise tandis que celle obtenue par la méthode itérative reste acceptable.
*Matrice associée à la déformation d&#8217;une corde élastique*\
On considère le système, de taille $n\times n$, $A\mathbf{u}=\mathbf{b}$ où


$$
\label{eq:ex4}
A=\begin{pmatrix}
\frac{2}{h}& -\frac{1}{h} &  0   & \cdots  &\cdots  & 0 \\
-\frac{1}{h}&\frac{2}{h}&-\frac{1}{h}&      &       &\vdots\\
0&-\frac{1}{h} &   \ddots  &\ddots&      &\\
\vdots &       &   \ddots  &      &-\frac{1}{h}&\vdots\\
\vdots &            &           &-\frac{1}{h}&\frac{2}{h}&-\frac{1}{h}\\
0&\cdots      &    \cdots       &0 & -\frac{1}{h} &\frac{2}{h}
\end{pmatrix} \quad \text{ et } \quad
\mathbf{b} = \begin{pmatrix}
h \\ h \\ \vdots \\ \ \\ \vdots\\ h \\ h \end{pmatrix}, \qquad h=\frac{1}{n+1}
$$

Comme on le verra plus loin dans le cours, la solution de ce système représente la déformation d&#8217;une corde élastique de longueur 1, soumise à une densité de force unitaire, aux points latexmath:[x_i = hi, \;
i=1,\ldots,n] et fixée aux extrémités $x=0$ et $x=1$.

Il est intéressant d&#8217;étudier le comportement du conditionnement de la matrice $A$ en fonction de la taille $n$.  Le conditionnement affectera les méthodes directes et itératives de façon différente.

Avec Python on peut facilement construire la matrice et calculer son conditionnement, avec la commande `cond`, pour différentes valeurs de $n$ (ex. $n=5,10,\ldots$) :


In [0]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
# Initialize empty list for storing condition numbers
K = []

# Loop equivalent to the Octave for-loop
for i in range(1, 11):  # attention on termine à 10, l'extremité droite est excluse
    n = 5 * i
    h = 1 / (n + 1)

    # Create the A matrix using numpy's diag function
    A = (2/h) * np.diag(np.ones(n)) - (1/h) * np.diag(np.ones(n-1), 1) \
        - (1/h) * np.diag(np.ones(n-1), -1)

    K.append(np.linalg.cond(A))

# Create a DataFrame
df = pd.DataFrame({
    'Matrix Size': list(range(5, 51, 5)),  # crée la liste [5, 10, ..., 50]
    'Condition Number (K)': K
})

# Affiche le tableau en format markdown
print(df.to_markdown())


On peut à présent tracer le conditionnement de la matrice en fonction de la taille $n$ de la matrice.


In [0]:
# Plotting with plotly
x_vals = list(range(5, 51, 5))  # This creates a list [5, 10, ..., 50]

fig = go.Figure(data=go.Scatter(x=x_vals, y=K, mode='lines+markers'))
fig.update_layout(title="Condition Number vs Matrix Size",
                  xaxis_title="Matrix Size",
                  yaxis_title="Condition Number")
fig.show()


*La matrice de Hilbert*\
La matrice de Hilbert de taille $n\times n$ est une matrice symétrique, définie par


$$
A_{ij} = \frac{1}{i+j-1}, \qquad i,j=1,\ldots,n
$$

Dans Matlab on peut construire une matrice de Hilbert de taille $n$ quelconque en utilisant la commande `hilb(n)`. Cette matrice est un exemple de matrice très mal conditionnée.

On considère les matrices de Hilbert de taille $n=4,6,8,10,12,14,\ldots$ et on construit des systèmes linéaires $A\mathbf{x} = \mathbf{b}$ dont la solution exacte $\mathbf{x}_{ex}$ est le vecteur unitaire et le terme de droite $\mathbf{b} = A\mathbf{x}_{ex}$. Pour chaque $n$, on calcule le conditionnement de la matrice, on résout le système linéaire par la factorisation $LU$ et on compare la solution obtenue avec la solution exacte du système.


In [0]:
import numpy as np
from scipy.linalg import hilbert
import plotly.graph_objects as go
import pandas as pd

# Initialize empty lists for storing values
nn = []
Acond = []
erreur_lu = []

# Loop equivalent to the Octave for-loop
for j in range(2, 8):  # Python's range is exclusive at the end
    n = 2 * j
    i = j - 1
    nn.append(n)

    # Generate the Hilbert matrix
    A = hilbert(n)

    x_ex = np.ones((n, 1))
    b = np.dot(A, x_ex)

    Acond.append(np.linalg.cond(A))

    # Solve the linear system
    x = np.linalg.solve(A, b)

    erreur_lu.append(np.linalg.norm(x - x_ex) / np.linalg.norm(x_ex))

# Create a DataFrame
df = pd.DataFrame({'nn': nn, 'Acond': Acond, 'erreur_lu': erreur_lu})

# Print the table in markdown format
print(df.to_markdown())


nous pouvons à présent tracer le conditionnement de la matrice et l&#8217;erreur relative en fonction de la taille $n$ de la matrice.


In [0]:
# Create the plot
fig = go.Figure()

# Add traces for Acond and erreur_lu
fig.add_trace(go.Scatter(x=nn, y=Acond, mode='lines', name='Acond'))
fig.add_trace(go.Scatter(x=nn, y=erreur_lu, mode='lines+markers', line=dict(dash='dash'), name='erreur_lu'))

# Set the Y-axis to a log scale
fig.update_layout(yaxis_type="log")

# Show the plot
fig.show()


La figure  montre le conditionnement obtenu ainsi que l&#8217;erreur $\| \mathbf x_{ex} - \mathbf x_{calc}\|/\| \mathbf  x_{ex}\|$ où $\|\cdot\|$ est la norme euclidienne d&#8217;un vecteur: $\|\mathbf{x}\| = \sqrt{\mathbf{x}^T\cdot\mathbf{x}}$.
Dans le même graphe on à montré aussi l&#8217;erreur obtenue en utilisant une méthode itérative (voir section suivante).
On remarque que lorsque le conditionnement de la matrice devient très grand, la solution obtenue par la méthode directe est très mauvaise tandis que celle obtenue par la méthode itérative reste acceptable.
